In [1]:
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# !pip install cmath
!pip install torch
!pip install sklearn
!pip install pandas
!pip install librosa
!pip install numpy
!pip install speechpy
!pip install transformers
!pip install jiwer
!pip install pyctcdecode

from pyctcdecode import build_ctcdecoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=fb8a99d75da85bb6550301b63693c9c0868b99d2e9ad65b55f3b8cd19337d8c0
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab

In [3]:
!pip install pytube
!pip install pydub
!pip install moviepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 7.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install imageio-ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 56.8 MB/s eta 0:00:00


In [5]:
cd /content/drive/MyDrive/PL

/content/drive/MyDrive/PL


In [6]:
# from model import Net
# from cProfile import label
# from infer import phonetic_embedding
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import torch.nn as nn
import torch.nn.functional as F
from phonetic_encoder import Phonetic_encoder
from acoustic_encoder import Acoustic_encoder
from linguistic_encoder import Linguistic_encoder
from char_embedding import text_to_tensor, clean_corpus
from help import wav_norm, Atention
import numpy as np
# from help import beam_search_decoding
from model import Acoustic_Phonetic_Linguistic
# from dataloader import MDD_Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from jiwer import wer, cer
tokenizer = Wav2Vec2Processor.from_pretrained("pretrained_finetuned")

net = Acoustic_Phonetic_Linguistic()





/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:375: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
cd /content

/content


In [8]:
import os
import argparse
import librosa
import soundfile as sf
import re
import numpy
from pytube import YouTube, Playlist
import pytube
import torch
import torchaudio
import torchaudio.functional as F
import subprocess


# from moviepy.editor import *
import glob
from pydub import AudioSegment

parser = argparse.ArgumentParser()
#download setting




# video index to continue crawling, index = 0 means the first video
START_INDEX = 0

# importing the module
from pytube import YouTube

from pytube import YouTube
yt = YouTube('https://www.youtube.com/watch?v=spdAiWwkBtE&ab_channel=AnhChillOfficial')
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution')[-1].download()



'/content/Thói Quen-Hoàng Dũng-GDucky Rap Cover.mp4'

In [9]:
x = glob.glob("*.mp4")
for i in range(len(x)):
  os.rename(x[0], 'test.mp4')

In [10]:
command = "ffmpeg -i test.mp4 -ab 160k -ac 2 -ar 44100 -vn /content/test.wav"
subprocess.call(command, shell=True)

i = 'test.wav'
y, sr = torchaudio.load(i)       
y_16k = F.resample(y, sr, 16000)
y_16k = y_16k.numpy()
y_mono = librosa.to_mono(y_16k)
sf.write(i, y_mono, 16000)

In [11]:
# from pydub import AudioSegment
# t1 = 26 * 1000 #Works in milliseconds
# t2 = 56 * 1000
# newAudio = AudioSegment.from_wav("test.wav")
# newAudio = newAudio[t1:t2]
# newAudio.export('test.wav', format="wav")

In [12]:
cd /content/drive/MyDrive/PL

/content/drive/MyDrive/PL


In [13]:
import torch
from infer import phonetic_embedding

net = torch.load('/content/drive/MyDrive/PL/MDD_Checkpoint/song_checkpoint.pth')
net.eval().to('cuda')
path = '/content/test.wav'
lyrics = 'Nửa năm từ khi em ra đi, anh chẳng khác là bao Thuốc thì không hút mấy, nhưng một khi đã hút là nát cả bao Bạn anh hỏi giải pháp là sao? Dạo này không thấy khát và khao? Anh trả lời dạo này chỉ muốn căn nhà 2 đứa có rap và tao ? Mà thôi ông ơi Về Bia rượu thì là tôi không chơi Cố nhắm mắt để ngủ sớm, sáng mai đi tập và sau đó thì là bơi, xông hơi Yeah im feeling so gud Trưa ăn cơm, ban đêm anh ăn soup Làm việc hàng ngày nhưng mà lười một chút Thiền định là vào tầm là mười một phút yeah yeah But u killin me god Yeah yeah Không còn em kế bên Tôi đang làm quen được bao điều hay nhưng sao tôi không thể quên But u killin me god Yeah yeah Không còn em kế bên Tôi đang làm quen được bao điều hay nhưng sao tôi không thể quên Làm sao quến bao đêm xem phim mì gói em nấu I’m fuckin fall in love. Dù cho nó là một thói quen xấu Sau bao nhiêu đêm tôi đã thao thức, vì còn một điều này cần phải giấu I still keep it rollin roll buông ra câu i love u so'
phonetic = phonetic_embedding(path).squeeze(0)
linguistic = text_to_tensor(lyrics)
linguistic = torch.tensor(linguistic)
phonetic = phonetic.to('cuda')
linguistic = linguistic.to('cuda')
phonetic = phonetic.unsqueeze(0)
linguistic = linguistic.unsqueeze(0)
outputs = net(phonetic, linguistic)
outputs = outputs.detach().cpu().numpy()

/content/drive/MyDrive/PL/linguistic_encoder.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float)


In [14]:
np.save("/content/output.npy", outputs)

In [15]:
def clean_corpus(str1):
    res1 = ""
    for i in str1:
        if i.isalpha() or i==" ":
            res1 = res1 + i
    return res1.lower().strip().replace("  ", " ")
print(lyrics)
print(clean_corpus(lyrics))

Nửa năm từ khi em ra đi, anh chẳng khác là bao Thuốc thì không hút mấy, nhưng một khi đã hút là nát cả bao Bạn anh hỏi giải pháp là sao? Dạo này không thấy khát và khao? Anh trả lời dạo này chỉ muốn căn nhà 2 đứa có rap và tao ? Mà thôi ông ơi Về Bia rượu thì là tôi không chơi Cố nhắm mắt để ngủ sớm, sáng mai đi tập và sau đó thì là bơi, xông hơi Yeah im feeling so gud Trưa ăn cơm, ban đêm anh ăn soup Làm việc hàng ngày nhưng mà lười một chút Thiền định là vào tầm là mười một phút yeah yeah But u killin me god Yeah yeah Không còn em kế bên Tôi đang làm quen được bao điều hay nhưng sao tôi không thể quên But u killin me god Yeah yeah Không còn em kế bên Tôi đang làm quen được bao điều hay nhưng sao tôi không thể quên Làm sao quến bao đêm xem phim mì gói em nấu I’m fuckin fall in love. Dù cho nó là một thói quen xấu Sau bao nhiêu đêm tôi đã thao thức, vì còn một điều này cần phải giấu I still keep it rollin roll buông ra câu i love u so
nửa năm từ khi em ra đi anh chẳng khác là bao thuốc

In [16]:
# /content/drive/MyDrive/output_alignment_fusion/3130303538355f3338.npy
# !pip install torch
# from torch.utils.data import Dataset
# import matplotlib.pyplot as plt
# import os
# import pandas as pd
import torch.nn.functional as F
import torch
import numpy as np
# import numpy as np
# from char_embedding import tensor_to_text
import matplotlib.pyplot as plt
import glob


x = np.load("/content/output.npy")

# print(x)

x = torch.tensor(x)
predicted_ids = torch.argmax(torch.tensor(x), dim=1)


x = torch.log_softmax(x, dim=-1)
x = x.cpu().detach()


labels = ('ắ', 'ồ', 'z', 'ứ', 'ỡ', 'ì', 'x', 'ặ', 'u', 'ẹ', 'd', 'ỵ', 'r', 'p', 't', 'ỳ', 'ẩ', 'f', 'ó', 'á', 'v', 'ã', 'i', 'ư', 'ở', 'ễ', 'ụ', 'ú', 'ũ', ' ', 'ă', 'é', 'ằ', 'a', 'ấ', 'ờ', 'ữ', 'ớ', 'n', 'ý', 's', 'h', 'ơ', 'ị', 'l', 'c', 'k', 'ỷ', 'ỗ', 'ế', 'ẻ', 'ợ', 'ẫ', 'í', 'ỏ', 'ủ', 'g', 'q', 'j', 'ò', 'ỹ', 'ự', 'ô', 'b', 'y', 'ĩ', 'ỉ', 'ẵ', 'ầ', 'ê', 'ộ', 'ậ', 'm', 'ń', 'o', 'ọ', 'đ', 'ẽ', 'ử', 'à', 'è', 'e', 'ẳ', 'ổ', 'ù', 'w', 'ả', 'ạ', 'â', 'ệ', 'ề', 'õ', 'ố', 'ể', 'ừ',)

# def clean_corpus(str1):
#     res1 = ""
#     for i in str1:
#         if i.isalpha() or i==" ":
#             res1 = res1 + i
#     return res1.lower().strip().replace("  ", " ")
transcript = " " + lyrics + ""
transcript = clean_corpus(transcript)
dictionary = {c: i for i, c in enumerate(labels)}

tokens = [dictionary[c] for c in transcript]


def get_trellis(x, tokens, blank_id=95):
    num_frame = x.size(0)
    num_tokens = len(tokens)

    # Trellis has extra diemsions for both time axis and tokens.
    # The extra dim for tokens represents <SoS> (start-of-sentence)
    # The extra dim for time axis is for simplification of the code.
    trellis = torch.empty((num_frame + 1, num_tokens + 1))
    trellis[0, 0] = 0
    trellis[1:, 0] = torch.cumsum(x[:, 0], 0)
    trellis[0, -num_tokens:] = -float("inf")
    trellis[-num_tokens:, 0] = float("inf")

    for t in range(num_frame):
        trellis[t + 1, 1:] = torch.maximum(
            # Score for staying at the same token
            trellis[t, 1:] + x[t, blank_id],
            # Score for changing to the next token
            trellis[t, :-1] + x[t, tokens],
        )
    return trellis


trellis = get_trellis(x, tokens)

from dataclasses import dataclass
@dataclass
class Point:
  token_index: int
  time_index: int
  score: float


def backtrack(trellis, emission, tokens, blank_id=95):
  # Note:
  # j and t are indices for trellis, which has extra dimensions 
  # for time and tokens at the beginning.
  # When refering to time frame index `T` in trellis,
  # the corresponding index in emission is `T-1`.
  # Similarly, when refering to token index `J` in trellis,
  # the corresponding index in transcript is `J-1`.
  j = trellis.size(1) - 1
  t_start = torch.argmax(trellis[:, j]).item()

  path = []
  for t in range(t_start, 0, -1):
    # 1. Figure out if the current position was stay or change
    # Note (again):
    # `emission[J-1]` is the emission at time frame `J` of trellis dimension.
    # Score for token staying the same from time frame J-1 to T.
    stayed = trellis[t-1, j] + emission[t-1, blank_id]
    # Score for token changing from C-1 at T-1 to J at T.
    changed = trellis[t-1, j-1] + emission[t-1, tokens[j-1]]

    # 2. Store the path with frame-wise probability.
    prob = emission[t-1, tokens[j-1] if changed > stayed else 0].exp().item()
    # Return token index and time index in non-trellis coordinate.
    path.append(Point(j-1, t-1, prob))

    # 3. Update the token
    if changed > stayed:
      j -= 1
      if j == 0:
        break
  else:
    raise ValueError('Failed to align')
  return path[::-1]

path = backtrack(trellis, x, tokens)
@dataclass
class Segment:
    label: str
    start: int
    end: int
    score: float
    

    def __repr__(self):
      return f"{self.start} --> {self.end} {self.label} \n"

    @property
    def length(self):
        return self.end - self.start


def merge_repeats(path):
    i1, i2 = 0, 0
    segments = []
    while i1 < len(path):
        while i2 < len(path) and path[i1].token_index == path[i2].token_index:
            i2 += 1
        score = sum(path[k].score for k in range(i1, i2)) / (i2 - i1)
        segments.append(
            Segment(
                transcript[path[i1].token_index],
                path[i1].time_index,
                path[i2 - 1].time_index + 1,
                score,
            )
        )
        i1 = i2
    return segments


segments = merge_repeats(path)

# Merge words
def merge_words(segments, separator=" "):
    words = []
    i1, i2 = 0, 0
    while i1 < len(segments):
        if i2 >= len(segments) or segments[i2].label == separator:
            if i1 != i2:
                segs = segments[i1:i2]
                word = "".join([seg.label for seg in segs])
                score = sum(seg.score * seg.length for seg in segs) / sum(seg.length for seg in segs)
                words.append(Segment(word, segments[i1].start*20, segments[i2 - 1].end*20, score))
            i1 = i2 + 1
            i2 = i1
        else:
            i2 += 1
    return words


word_segments = merge_words(segments)


<ipython-input-16-0cd433317fd5>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_ids = torch.argmax(torch.tensor(x), dim=1)


In [17]:
def format_millis(milliseconds):
  millis = milliseconds
  millisecond = int(millis%1000)
  millisecond = str(millisecond)
  if len(millisecond)==0:
    millisecond = '000'
  elif len(millisecond) == 1:
    millisecond = '00' + millisecond
  elif len(millisecond) == 2:
    millisecond = '0' + millisecond

  seconds=(millis/1000)%60
  seconds = int(seconds)
  seconds = str(seconds)
  if len(seconds) == 1:
    seconds = '0' + seconds
  minutes=(millis/(1000*60))%60
  
  minutes = int(minutes)
  minutes = str(minutes)
  if len(minutes) == 1:
    minutes = '0' + minutes

  hours=(millis/(1000*60*60))%24
  hours = int(hours)
  hours = str(hours)
  if len(hours) == 1:
    hours = '0' + hours

  return (hours + ":" + minutes + ":" +  seconds + "," + millisecond)

In [21]:
f = open("/content/rap_thoiquen.srt", "a")
for i in range(len(word_segments)):
 
  if (i==0):
    start = str(str(word_segments[i])).split(" ")[0]
    end_before = int(str(str(word_segments[i])).split(" ")[2])
    word = str(str(word_segments[i])).split(" ")[3]
    if int(start)>=200:
      start = int(start)-200
    else:
      start = 0
    start_next = int(str(word_segments[i+1]).split(" ")[0])
    end = (2*end_before + start_next)/3
  elif i==len(word_segments)-1:
    end_before = int(str(word_segments[i-1]).split(" ")[2])
    start_next = int(str(word_segments[i]).split(" ")[0])
    end = int(str(word_segments[i]).split(" ")[2]) + 250
    start = (2*end_before+start_next)/3
    word = str(word_segments[i]).split(" ")[3]
  else:
    end_before = int(str(word_segments[i-1]).split(" ")[2])
    start_next = int(str(word_segments[i+1]).split(" ")[0])
    end = int(str(word_segments[i]).split(" ")[2])
    start = int(str(word_segments[i]).split(" ")[0])
    start = (2*end_before+start)/3
    end = (2*end + start_next)/3
    word = str(word_segments[i]).split(" ")[3]
  
  start = (format_millis(start))
  end = (format_millis(end))
  one_line = str(i+1) + "\n" + start + " --> " + end + "\n" + str(word) + "\n\n"
  print(one_line)
  f.write(one_line)


1
00:00:00,000 --> 00:00:01,146
nửa


2
00:00:01,146 --> 00:00:01,286
năm


3
00:00:01,286 --> 00:00:01,446
từ


4
00:00:01,446 --> 00:00:01,566
khi


5
00:00:01,566 --> 00:00:01,666
em


6
00:00:01,666 --> 00:00:01,853
ra


7
00:00:01,853 --> 00:00:02,646
đi


8
00:00:02,646 --> 00:00:02,886
anh


9
00:00:02,886 --> 00:00:03,086
chẳng


10
00:00:03,086 --> 00:00:03,346
khác


11
00:00:03,346 --> 00:00:03,500
là


12
00:00:03,500 --> 00:00:04,406
bao


13
00:00:04,406 --> 00:00:04,666
thuốc


14
00:00:04,666 --> 00:00:04,826
thì


15
00:00:04,826 --> 00:00:05,033
không


16
00:00:05,033 --> 00:00:05,233
hút


17
00:00:05,233 --> 00:00:05,426
mấy


18
00:00:05,426 --> 00:00:05,566
nhưng


19
00:00:05,566 --> 00:00:05,653
một


20
00:00:05,653 --> 00:00:05,946
khi


21
00:00:05,946 --> 00:00:06,100
đã


22
00:00:06,100 --> 00:00:06,366
hút


23
00:00:06,366 --> 00:00:06,540
là


24
00:00:06,540 --> 00:00:06,780
nát


25
00:00:06,780 --> 00:00:06,953
cả


26
00:00:06,953 --> 00:00:07,366


In [19]:
# !ffmpeg -i /content/lyric_align.jpg -i "/content/test.wav" -sub_charenc utf-8 -i "/content/yeuyeuyeu.srt" "/content/OUTPUT.mp4"

In [20]:
!ffmpeg -i /content/test.mp4 -sub_charenc utf-8 -i "/content/rap_thoiquen.srt" "/content/OUTPUT.mp4"

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e